In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset


pic_dataset = load_dataset("PiC/phrase_similarity")


train_data = pic_dataset['train']
validation_data = pic_dataset['validation']
test_data = pic_dataset['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
import gensim
import keras
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [ ]:
phrase_1 = validation_data['phrase1']

phrase_2 = validation_data['phrase2']


In [ ]:
import numpy as np
from transformers import BertTokenizer, TFBertModel

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained("bert-base-cased",output_hidden_states=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf

In [ ]:

# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
# model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


In [ ]:
import tensorflow as tf
# from transformers import BertTokenizer, TFBertModel

# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# model = TFBertModel.from_pretrained("bert-base-cased", output_hidden_states=True)

def find_embedding_g(sentences):
    encoded_input = tokenizer(sentences,padding=True, truncation=True, return_tensors='tf' )
    model_output = model(encoded_input)
    token_embeddings = model_output.last_hidden_state

    res = []
    for token_embedding in token_embeddings:
        attention_mask = tf.cast(encoded_input['attention_mask'], tf.float32)
        masked_embeddings = token_embedding * tf.expand_dims(attention_mask, axis=-1)
        averaged_embeddings = tf.reduce_sum(masked_embeddings, axis=1) / tf.maximum(tf.reduce_sum(attention_mask, axis=1, keepdims=True), 1e-9)
        res.append(averaged_embeddings)
    return res




In [ ]:
sentence_s = ['Hi this me the sample text of testing the same', 'hi this is me']
li = find_embedding_g(sentence_s)
print(li[0].shape)

(2, 768)


In [ ]:
em_1 = find_embedding_g(phrase_1)
em_2 = find_embedding_g(phrase_2)

In [ ]:
# encoded_input = tokenizer(phrase_1[:2000],padding=True, truncation=True, return_tensors='tf' )
# model_output = model(encoded_input)
# token_embeddings = model_output.last_hidden_state


In [ ]:
# res = []
# for token_embedding in token_embeddings:
#     attention_mask = tf.cast(encoded_input['attention_mask'], tf.float32)
#     masked_embeddings = token_embedding * tf.expand_dims(attention_mask, axis=-1)
#     averaged_embeddings = tf.reduce_sum(masked_embeddings, axis=1) / tf.maximum(tf.reduce_sum(attention_mask, axis=1, keepdims=True), 1e-9)
#     res.append(averaged_embeddings)
# print(res[0].shape)

In [ ]:
print(len(em_1),len(em_2))

1000 1000


In [ ]:
l = len(em_1[0])
print(l)

1000


Sentence embeddings:
tensor([[-0.3931,  0.0389,  1.9874,  ..., -0.6094, -1.0946,  0.3265],
        [ 0.0615,  0.3274,  1.8332,  ..., -0.1299,  0.4609,  0.2404],
        [-0.2765,  0.3784,  2.0411,  ..., -0.2447,  0.1843,  0.2362]])

In [ ]:
cos_similarities = []

for vec1, vec2 in zip(em_1, em_2):

    vec1 = np.squeeze(vec1.numpy())
    vec2 = np.squeeze(vec2.numpy())

    # Compute cosine similarity
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)

    # Append the cosine similarity to the list
    cos_similarities.append(cosine_sim)

ValueError: shapes (1000,768) and (1000,768) not aligned: 768 (dim 1) != 1000 (dim 0)

In [ ]:
v_l = validation_data['label']
cnt = 0
for i in range(len(v_l)):
    if(v_l[i]==1):
        cnt+=1
cnt

500

In [ ]:
cos_similarities[0].shape

(1000,)

In [ ]:
per_w = 0
ccc = cos_similarities.copy()

for i in range(len(ccc)):
    if(cos_similarities[i]>0.913):
        ccc[i] = 1
    else:
        ccc[i] = 0
for i in range(len(ccc)):
    if(ccc[i]==v_l[i]):
        per_w+=1
print(per_w)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:

def find_embedding_c(sentences, phrases):
    contextualized_embeddings_2 = []
    for sentence, phrase in zip(sentences, phrases):

        encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='tf')
        token_phrase = tokenizer.tokenize(phrase)
        tokenized_sentence = tokenizer.tokenize(sentence)


        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)
        token_embeddings =  model_output[0] # of all tokens
        # print(token_embeddings.shape)
        start_index = None
        end_index = None
        # print(tokenized_sentence)
        # print(len(tokenized_sentence))
        for i in range(len(tokenized_sentence)):
            # print(tokenized_sentence[i:i+len(token_phrase)])
            # print(token_phrase)
            if tokenized_sentence[i:i+len(token_phrase)] == token_phrase:
                start_index = i
                end_index = i + len(token_phrase)
                break
        if(start_index is None or end_index is None):
            print(tokenized_sentence,token_phrase)
            return []
        start_index+=1
        end_index+=1
        # print(tokenized_sentence)
        # print(token_phrase)
        # print(start_index,end_index)

        last_hidden_states_np = model_output.last_hidden_state.numpy()
        phrase_embeddings = last_hidden_states_np[:, start_index:end_index, :]
        phrase_embedding_mean = np.mean(phrase_embeddings, axis=1)
        contextualized_embeddings_2.append(phrase_embedding_mean)
        # contextualized_embeddings_2.append(find_average(token_embeddings,  encoded_input['attention_mask']))
    return contextualized_embeddings_2




In [ ]:
sentences = ['This is an example sentence', 'Each sentence is converted', 'i am r of this worldy genius']
phrases = [' ;is an example', 'sentence','r of' ]
li = find_embedding_c(sentences, phrases)
print(len(li))



['This', 'is', 'an', 'example', 'sentence'] [';', 'is', 'an', 'example']
0


In [ ]:
sentences = ['This is an example sentence', 'Each sentence is converted', 'i am imotr of this worldy genius','Around 1906, Menelik became incapacitated and "Itege" Taytu Betul became the "de facto" power behind the throne.']
phrases = ['is an example', 'sentence','this worldy','"de facto" power' ]
li = find_embedding_c(sentences, phrases)
print(len(li))

4


In [ ]:
sentence_1 = validation_data['sentence1']
sentence_2 = validation_data['sentence2']

In [ ]:
phrase_1 = validation_data['phrase1']
phrase_2 = validation_data['phrase2']
sentence_1 = validation_data['sentence1']
sentence_2 = validation_data['sentence2']
emc_1 = find_embedding_c(sentence_1, phrase_1)
emc_2 = find_embedding_c(sentence_2, phrase_2)
print(len(emc_1), len(emc_2))

KeyboardInterrupt: 

In [ ]:
print(emc_1[0].shape)

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
cos_similarities = []


for vec1, vec2 in zip(em_1, em_2):

    vec1 = vec1.squeeze()
    vec2 = vec2.squeeze()

    # Compute cosine similarity
    cosine_sim = F.cosine_similarity(vec1, vec2, dim=0)

    # Append the cosine similarity to the list
    cos_similarities.append(cosine_sim.item())

In [ ]:
emc_1_tensors = [torch.tensor(arr) for arr in emc_1]
emc_2_tensors = [torch.tensor(arr) for arr in emc_2]


cos_c_similarities = []


for vec1, vec2 in zip(emc_1_tensors, emc_2_tensors):

    vec1 = vec1.squeeze()  # Remove the first dimension
    vec2 = vec2.squeeze()  # Remove the first dimension


    cosine_sim = F.cosine_similarity(vec1, vec2, dim=0)


    cos_c_similarities.append(cosine_sim.item())



In [ ]:
# for i in range(len(cos_similarities)):
#     print(cos_c_similarities[i], cos_similarities[i])


In [ ]:
v_l = train_data['label'][:2000]
cnt = 0
for i in range(len(v_l)):
    if(v_l[i]==1):
        cnt+=1
cnt

In [ ]:
per = 0
ccc = cos_c_similarities.copy()

for i in range(len(ccc)):
    if(cos_c_similarities[i]>0.913):
        ccc[i] = 1
    else:
        ccc[i] = 0
o_c = 0

# for i in range(len(ccc)):
#     if(ccc[i]==1):
#         o_c+=1
# print(o_c)
for i in range(len(ccc)):
    if(ccc[i]==v_l[i]):
        per+=1
    # else:
    #     per-=1
print(per)

Hence a 60.7% chance with cosine similarity with context

In [ ]:
per_w = 0
ccc = cos_similarities.copy()

for i in range(len(ccc)):
    if(cos_similarities[i]>0.913):
        ccc[i] = 1
    else:
        ccc[i] = 0
for i in range(len(ccc)):
    if(ccc[i]==v_l[i]):
        per_w+=1
print(per_w)

Hence a 52.4% chance with cosine similarity without contex

Now Let's train a deep neural network for the same

In [ ]:
sentence_1 = train_data['sentence1'][:2000]
phrase_3 = train_data['phrase1'][:2000]
sentence_2 = train_data['sentence2'][:2000]
phrase_4 = train_data['phrase2'][:2000]

In [ ]:
print(em_1[0][0][0], em_2[0][0][0]) # they aren't same

In [ ]:
em_3 = em_1.copy()
em_4 = em_2.copy()

In [ ]:
emc_3 = emc_1.copy()
emc_4 = emc_2.copy()

In [ ]:
print(len(em_3), len(em_4), len(emc_3), len(emc_4))
print(em_1[0][0][0] ,em_3[0][0][0])

In [ ]:
em_5 = [torch.cat((t3, t4), dim=1).squeeze() for t3, t4 in zip(em_3, em_4)]
em_5[0].shape
len(em_5)

In [ ]:
import tensorflow.keras as keras


predictor = keras.models.Sequential([
    keras.layers.Dense(256, input_shape=(1536,), activation='relu'),  # ReLU layer with 256 units
    keras.layers.Dense(1, activation='sigmoid')  # 1-output linear classification layer with sigmoid activation
])


predictor.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
col_to_predict =np.ravel( train_data['label'][:2000])
len(em_5[0])
em_5 = np.array(em_5)


In [ ]:
em_5.shape

In [ ]:
predictor.fit(em_5, col_to_predict,epochs = 100)

In [ ]:
col_test = test_data['label']
test_p1 = test_data['phrase1']
test_p2 = test_data['phrase2']
test_s1 = test_data['sentence1']
test_s2 = test_data['sentence2']

In [ ]:
emt_1 = find_embedding_g(test_p1)
emt_2 = find_embedding_g(test_p2)

In [ ]:
# COSINE SIMILARITY ON TEST SET
cos_similarities_t = []


for vec1, vec2 in zip(emt_1, emt_2):

    vec1 = vec1.squeeze()
    vec2 = vec2.squeeze()

    # Compute cosine similarity
    cosine_sim = F.cos_similarities_t(vec1, vec2, dim=0)

    # Append the cosine similarity to the list
    cos_similarities_t.append(cosine_sim.item())


In [ ]:
v_l = test_data['label']
cnt = 0
for i in range(len(v_l)):
    if(v_l[i]==1):
        cnt+=1
cnt

In [ ]:
per_w = 0
ccc = cos_similarities_t.copy()

for i in range(len(ccc)):
    if(cos_similarities_t[i]>0.913):
        ccc[i] = 1
    else:
        ccc[i] = 0
for i in range(len(ccc)):
    if(ccc[i]==v_l[i]):
        per_w+=1
print(per_w)

In [ ]:
print(len(emt_1), len( emt_2))

In [ ]:
emt_3 = [torch.cat((t3, t4), dim=1).squeeze() for t3, t4 in zip(emt_1, emt_2)]
emt_3 = np.array(emt_3)
emt_3.shape

In [ ]:
len(col_test)

In [ ]:

predictor.evaluate(emt_3, np.ravel(col_test), verbose = 1)

So without Context we get 44% which is random chance

In [ ]:
emct_1 = find_embedding_c(test_s1, test_p1)
emct_2 = find_embedding_c(test_s2, test_p2)

In [ ]:
print(type(emct_1[0]), emct_1[0].shape)

In [ ]:
emct_f = np.concatenate((emct_1, emct_2), axis=1).reshape(-1, 1536)
emct_f.shape

In [ ]:
emc_3 = emc_3
emc_4 = emc_4

In [ ]:
cols_to_predict = np.ravel(train_data['label'])

In [ ]:
emc_f  = np.concatenate((emc_3, emc_4), axis=1).reshape(-1, 1536)
emc_f.shape

In [ ]:
# now i have to train on emc_f and predict cols_to_predict
predictor = keras.models.Sequential([
    keras.layers.Dense(256, input_shape=(1536,), activation='relu'),  # ReLU layer with 256 units
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # 1-output linear classification layer with sigmoid activation
])


predictor.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
predictor.fit(emc_f, cols_to_predict,epochs = 100)

In [ ]:
predictor.evaluate(emct_f,np.ravel(col_test), verbose = 1)

So there in increment in the accuracy when we check phrase sense with context